In [4]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Cloning into 'Lifespan_Age_Transformation_Synthesis'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 618 (delta 4), reused 12 (delta 4), pack-reused 604 (from 1)
Receiving objects: 100% (618/618), 22.86 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/Lifespan_Age_Transformation_Synthesis/Lifespan_Age_Transformation_Synthesis/Lifespan_Age_Transformation_Synthesis


In [5]:
!pip3 install -r requirements.txt


In [6]:
!pip install Pillow==9.4.0  # Use a version < 10.0.0 to avoid ANTIALIAS issue


In [7]:
!python download_models.py

  0% 2.38k/213M [00:00<19:08:21, 3.09kB/s]Google Drive download failed.
Trying do download from alternate server

  0% 0.00/213M [00:00<?, ?B/s]
  0% 0.00/213M [00:00<?, ?B/s]
  0% 131k/213M [00:01<27:07, 131kB/s]
  0% 262k/213M [00:01<13:47, 257kB/s]
  0% 524k/213M [00:01<06:32, 542kB/s]
  0% 918k/213M [00:01<03:37, 977kB/s]
  1% 1.84M/213M [00:01<01:39, 2.13MB/s]
  2% 3.67M/213M [00:01<00:46, 4.51MB/s]
  3% 6.16M/213M [00:01<00:27, 7.43MB/s]
  4% 8.78M/213M [00:02<00:20, 9.81MB/s]
  5% 11.3M/213M [00:02<00:17, 11.3MB/s]
  7% 13.9M/213M [00:02<00:15, 12.7MB/s]
  8% 16.4M/213M [00:02<00:14, 13.4MB/s]
  9% 19.0M/213M [00:02<00:13, 14.1MB/s]
 10% 21.6M/213M [00:02<00:13, 14.6MB/s]
 11% 24.1M/213M [00:03<00:12, 14.8MB/s]
 13% 26.7M/213M [00:03<00:12, 15.1MB/s]
 14% 29.2M/213M [00:03<00:12, 15.2MB/s]
 15% 31.9M/213M [00:03<00:11, 15.3MB/s]
 16% 34.3M/213M [00:03<00:11, 15.4MB/s]
 17% 37.0M/213M [00:03<00:11, 15.4MB/s]
 19% 39.6M/213M [00:04<00:11, 15.6MB/s]
 20% 42.1M/213M [00:04<00:11, 15

In [8]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes


------------ Options -------------
activation: lrelu
batchSize: 1
checkpoints_dir: ./checkpoints
compare_to_trained_class: 1
compare_to_trained_outputs: False
conv_weight_norm: True
dataroot: ./datasets/males/
debug_mode: False
decoder_norm: pixel
deploy: False
display_id: 1
display_port: 8097
display_single_pane_ncols: 6
display_winsize: 256
fineSize: 256
full_progression: False
gen_dim_per_style: 50
gpu_ids: [0]
how_many: 50
id_enc_norm: pixel
image_path_file: None
in_the_wild: False
input_nc: 3
interp_step: 0.5
isTrain: False
loadSize: 256
make_video: False
max_dataset_size: inf
nThreads: 4
n_adaptive_blocks: 4
n_downsample: 2
name: debug
ngf: 64
no_cond_noise: False
no_flip: False
no_moving_avg: False
normalize_mlp: True
ntest: inf
output_nc: 3
phase: test
random_seed: -1
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
sort_classes: True
sort_order: ['0-2', '3-6', '7-9', '15-19', '30-39', '50-69']
trained_class_jump: 1
traverse: False
use_modulated_con

usage: colab_kernel_launcher.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                                [--checkpoints_dir CHECKPOINTS_DIR] [--batchSize BATCHSIZE]
                                [--loadSize LOADSIZE] [--fineSize FINESIZE] [--input_nc INPUT_NC]
                                [--output_nc OUTPUT_NC] [--dataroot DATAROOT]
                                [--sort_classes SORT_CLASSES] [--sort_order SORT_ORDER]
                                [--resize_or_crop RESIZE_OR_CROP] [--serial_batches] [--no_flip]
                                [--nThreads NTHREADS] [--max_dataset_size MAX_DATASET_SIZE]
                                [--display_single_pane_ncols DISPLAY_SINGLE_PANE_NCOLS]
                                [--display_winsize DISPLAY_WINSIZE] [--display_port DISPLAY_PORT]
                                [--display_id DISPLAY_ID]
                                [--use_modulated_conv USE_MODULATED_CONV]
                                [--conv_weight_norm CONV_WEIGHT_NO

In [9]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

AgingDataLoader
dataset [MulticlassUnalignedDataset] was created


In [27]:
# Load both models
opt.name = 'males_model'
male_model = create_model(opt)
male_model.eval()

opt.name = 'females_model'
female_model = create_model(opt)
female_model.eval()

Generator(
  (id_encoder): IdentityEncoder(
    (encoder): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): EqualConv2d(
        (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      )
      (2): PixelNorm()
      (3): ReLU(inplace=True)
      (4): ReflectionPad2d((1, 1, 1, 1))
      (5): EqualConv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (6): PixelNorm()
      (7): ReLU(inplace=True)
      (8): ReflectionPad2d((1, 1, 1, 1))
      (9): EqualConv2d(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
      )
      (10): PixelNorm()
      (11): ReLU(inplace=True)
      (12): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): EqualConv2d(
            (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          )
          (2): PixelNorm()
          (3): ReLU(inplace=True)
          (4): ReflectionPad2d((1, 1, 1, 1))
          (5): EqualConv2d(

InferenceModel(
  (netG): Generator(
    (id_encoder): IdentityEncoder(
      (encoder): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): EqualConv2d(
          (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
        )
        (2): PixelNorm()
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): EqualConv2d(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
        )
        (6): PixelNorm()
        (7): ReLU(inplace=True)
        (8): ReflectionPad2d((1, 1, 1, 1))
        (9): EqualConv2d(
          (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
        )
        (10): PixelNorm()
        (11): ReLU(inplace=True)
        (12): ResnetBlock(
          (conv_block): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): EqualConv2d(
              (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
            )
            (2): PixelNorm()
            (3): ReLU(inpl

In [28]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Saving input.jpg to input.jpg
User uploaded file "input.jpg"


In [29]:
# Ensure img_path is valid
if not os.path.isfile(img_path):
    raise ValueError(f"Uploaded file '{img_path}' does not exist.")

In [30]:
# Attempt to get the data from the dataset
try:
    data = dataset.dataset.get_item_from_path(img_path)  # Ensure this method exists
except AttributeError as e:
    raise RuntimeError("Failed to get item from dataset. Check if 'get_item_from_path' exists.") from e
except Exception as e:
    raise RuntimeError("An error occurred while retrieving data from the dataset.") from e

# Determine which model to use based on the filename or user input
if "female" in img_path.lower():  # Example condition to check for female images
    model = female_model
else:
    model = male_model

# Perform inference using the selected model
try:
    visuals = model.inference(data)  # Ensure this method is valid and data is in the correct format
except Exception as e:
    raise RuntimeError("An error occurred during model inference.") from e

# Create the results directory if it doesn't exist
os.makedirs('results', exist_ok=True)

In [31]:
# Construct the output path for the video
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')

# Create a video from the visuals
try:
    visualizer.make_video(visuals, out_path)  # Ensure this method is valid
except Exception as e:
    raise RuntimeError("An error occurred while creating the video.") from e

In [32]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the
# result to webm for display purposes

!pip3 install webm
webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
!webm -i $out_path $webm_out_path
use_webm = True
from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

ffmpeg -hide_banner -i results/input.mp4 -pass 1 -passlogfile /tmp/tmpbjnba9b5 -c:v libvpx-vp9 -speed 4 -tile-columns 6 -frame-parallel 0 -row-mt 1 -b:v 0 -threads 2 -auto-alt-ref 1 -lag-in-frames 25 -g 128 -pix_fmt yuv420p -crf 25 -an -sn -y -f webm /dev/null
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'results/input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2mp41
    encoder         : Lavf59.27.100
  Duration: 00:00:05.05, start: 0.000000, bitrate: 207 kb/s
  Stream #0:0(und): Video: mpeg4 (Simple Profile) (mp4v / 0x7634706D), yuv420p, 256x256 [SAR 1:1 DAR 1:1], 205 kb/s, 20 fps, 20 tbr, 10240 tbn, 20 tbc (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (mpeg4 (native) -> vp9 (libvpx-vp9))
Press [q] to stop, [?] for help
[libvpx-vp9 @ 0x5b5e0149b680] v1.11.0
Output #0, webm, to '/dev/null':
  Metadata:
    major_brand     : isom
    minor_versi

In [33]:
files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>